In [1]:
!pip install bertopic
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install --upgrade joblib==1.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
from bertopic import BERTopic
import pandas as pd

In [16]:
df = pd.read_excel("/content/drive/MyDrive/TopicModeling/input_clean_comments_no_name_version2.xlsx")

In [17]:
docs = []
for index, row in df.iterrows():
    docs.append(str(row['merged_string_for_topic']))

In [18]:
topic_model = BERTopic(language='english',verbose=True)
topics, probs = topic_model.fit_transform(docs)

Batches:   0%|          | 0/1625 [00:00<?, ?it/s]

2023-01-06 23:32:00,291 - BERTopic - Transformed documents to Embeddings
2023-01-06 23:33:37,003 - BERTopic - Reduced dimensionality
2023-01-06 23:33:42,906 - BERTopic - Clustered reduced embeddings


In [27]:
topic_model.visualize_topics()

Output hidden; open in https://colab.research.google.com to view.

In [28]:
new_df_topic_data = topic_model.get_topics()

In [29]:
new_df = pd.DataFrame(new_df_topic_data)

In [30]:
new_df.T.to_excel('/content/drive/MyDrive/TopicModeling/output_Bert_topic_many_withStopwords_topics_v2.xlsx')


In [19]:
topic_df = topic_model.get_topic_info()
topic_df.to_excel("/content/drive/MyDrive/TopicModeling/output_Bert_topic_withStopwords_topic_info_v2.xlsx")

In [20]:
topic_model.get_topics()

{-1: [('this', 0.001671320848730026),
  ('the', 0.0016523558969673573),
  ('is', 0.001631061706359853),
  ('to', 0.0016211796153092696),
  ('and', 0.0016065996280228742),
  ('you', 0.0016030688165155416),
  ('we', 0.0015866401260897106),
  ('it', 0.0015683875163322852),
  ('have', 0.0015411182032967763),
  ('not', 0.0015264619286414214)],
 0: [('ibm', 0.012433857240575196),
  ('password', 0.0056032094270034374),
  ('setup', 0.0051961141223587646),
  ('reset', 0.00435539623200177),
  ('id', 0.004053436227918789),
  ('book', 0.004009222978216411),
  ('etr', 0.003337189710584333),
  ('ln', 0.003308429792314031),
  ('user', 0.003291509414346246),
  ('note', 0.003080958904351432)],
 1: [('reset', 0.021874726331139484),
  ('self', 0.01912536977000025),
  ('password', 0.01603356579195204),
  ('unlock', 0.01387736959981774),
  ('sap', 0.01256674642413698),
  ('selfservice', 0.010428907471546489),
  ('de6', 0.010336534905678303),
  ('name21889', 0.008412277180480836),
  ('myid', 0.0070801953434

In [22]:
df["topic"]=topics
df["'probs"]=probs
df.to_excel("/content/drive/MyDrive/TopicModeling/output_bert_topics_df_withStopwords_v2.xlsx")

In [23]:
topic_model.visualize_topics(nr)


NameError: ignored

In [ ]:
new_topics, new_probs = topic_model.reduce_topics(docs, topics, probs, nr_topics=500)

In [31]:
topic_model.visualize_hierarchy()

In [ ]:
topic_model.embedding_model

In [ ]:
from umap import UMAP
from sentence_transformers import SentenceTransformer
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = sentence_model.encode(docs, show_progress_bar=True)
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)


Batches:   0%|          | 0/2100 [00:00<?, ?it/s]

In [ ]:
topic_model.visualize_documents(docs, reduced_embeddings)

TypeError: ignored

In [24]:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

In [25]:
# Preprocess Documents
documents = pd.DataFrame({"Document": docs,
                          "ID": range(len(docs)),
                          "Topic": topics})
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

# Extract vectorizer and analyzer from BERTopic
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

# Extract features for Topic Coherence evaluation
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic) if words!=''] 
               for topic in range(len(set(topics))-1)]

# Evaluate
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [26]:
coherence

0.7203648391129405